In [1]:
# Make 2 sequences for LSTM model generation:
#     1. Note/Chord/Rest Sequence 
#     2. Duration Sequence 

In [148]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import math
import fractions
import decimal
import re

from music21 import converter, corpus, instrument, midi, note, chord, pitch, musicxml

In [110]:
# Base function that opens the midi file
def open_midi(path):
    mf = midi.MidiFile()
    mf.open(path)
    mf.read()
    mf.close()
    return midi.translate.midiFileToStream(mf)

# Reference function 
# TODO: Delete this thing later when you can rewrite it!!
def extract_notes(midi_part):
    notes = [] 
    for nt in midi_part.flat.notes:
        notes.append(nt)
    return notes


def get_offsets_durations(midi_part):
    parent_element = [] 
    ret = [] 
    offsets = []
    durations = [] 
    for nt in midi_part.flat.notes:
        offsets.append(nt.offset)
        durations.append(nt.duration.quarterLength)
    return offsets, durations


# Convert midi part to a numpy array
def build_track(midi_part):
    return None 

# Build a map of all notes and chords to a value in 
def build_note_tbl(midi_stream):
    
    tbl = {}
    count = 1
    
    tbl["REST"] = 0
    

    for part in midi_stream: 
        for nt in part.flat.notes:
            if (isinstance(nt, note.Note)):
                if (nt.pitch.nameWithOctave not in tbl):
                    tbl[nt.pitch.nameWithOctave] = count
                    count += 1 
            elif (isinstance(nt, chord.Chord)):
                chord_name = ""
                for i in nt.notes:
                    chord_name += i.pitch.nameWithOctave + " "
                chord_name = chord_name[0:len(chord_name)-1]
                if (chord_name not in tbl):
                    tbl[chord_name] = count
                    count += 1
    
    # tbl maps note/chord to int
    # inv_tbl maps int to note/chord 
    inv_tbl = {v: k for k, v in tbl.items()}
    
    return tbl, inv_tbl

# Coonvert the times from floats to fractions
def convert_to_frac(times):
    dur = [0 for i in range(len(times))]
    for i in range(len(times)):
        if not isinstance(times[i], float):
            val = times[i]
        else:
            val = fractions.Fraction(times[i])
        dur[i]= val
        
    return dur

# Get LCM of denominators
# vals is an array of fractions
def arr_lcm(vals): 
    arr = [0 for i in range(len(vals))]
    for i in range(len(vals)):
        arr[i] = vals[i].denominator 
    ans = np.lcm(arr[0], arr[1])
    for i in range(2,len(arr)):
        ans = np.lcm(ans, arr[i])
    return ans



In [111]:
midi_name = "Paco_de_Lucia_Entre_dos_Aguas.mid" 
track = open_midi(midi_name)

In [129]:
off, dur = get_offsets_durations(track[0])
off, dur = convert_to_frac(off), convert_to_frac(dur)


In [130]:
# Find out the smallest part of a bar we have to break this thing into
# Each quarter has to be broken into 'lcm' parts

lcm = arr_lcm(off)
off = list(map(lambda x : int(x*lcm), off))
dur = list(map(lambda x : int(x*lcm), dur))

In [132]:
tbl, inv_tbl = build_note_tbl(track)
note_objs = extract_notes(track[0])
notes = [0 for i in range(len(note_objs))]

for i in range(len(note_objs)):
    nt = note_objs[i]
    if (isinstance(nt, note.Note)):
        notes[i] = nt.pitch.nameWithOctave
    elif (isinstance(nt, chord.Chord)):
        chord_name = ""
        for a in nt.notes:
            chord_name += a.pitch.nameWithOctave + " "
        chord_name = chord_name[0:len(chord_name)-1]
        notes[i] = chord_name

note_fin = ['a' for i in range(len(note_objs))]


# Convert to note objects
for i in range(len(notes)):
    note_fin[i] = tbl[notes[i]]
    
notes = note_fin
del note_objs
del note_fin

In [ ]:
# Some chords aren't getting picked up by the 

In [161]:
# Final sequences to be ingested by the LSTM

note_seq = []
dur_seq = []

accum = 0 

for i in range(len(notes)-1):
    if (off[i+1] == off[i]):  
        new_dur = max(dur[i], dur[i+1])
        new_note = inv_tbl[notes[i]] + " " + inv_tbl[notes[i+1]]
        chord_name = new_note.split()
        chord_name.sort()
        new_note = ' '.join(chord_name)
    
        
#     if (accum == off[i]):
#         note_seq.append(notes[i])
#         dur_seq.append(dur[i])
#     else:
#         note_seq.append(0)
#         dur_seq.append(off[i] - accum)
#         accum += off[i] - accum
        
# print(dur_seq)


C4 E4 G3
A3 D4 F#4
C4 E4 G3
A3 D4 F#4
F#4 G4
A4 G4
A4 G4
F#4 G4
C4 D4
B3 E4
C4 E3 E4 G3
A2 C4 E3 G3
A3 D4 F#3
B3 E4 G3
A3 D4 F#3
B3 E4 G3
C5 D5


In [163]:
for i in range(len(off)-1):
    if (off[i+1] <= off[i]):
        print(i)
        print(i+1)
        print("SHIT")
        print()

0
1
SHIT

2
3
SHIT

4
5
SHIT

6
7
SHIT

45
46
SHIT

65
66
SHIT

78
79
SHIT

91
92
SHIT

182
183
SHIT

248
249
SHIT

255
256
SHIT

269
270
SHIT

958
959
SHIT

963
964
SHIT

968
969
SHIT

973
974
SHIT

1028
1029
SHIT

